## Parsing semi-structured text

The goal of this task is to extract relevant information from emergency services dispatch messages so they can be analysed for trends.

[Cfsscan.com](https://www.cfsscan.com/livefeed) publishes emergency service messages in near real time, along with a timestamp, message id and an agency identifier. Hundreds of messages are published each day, so extracting by hand is not feasible. Screenshot below shows how these messages are published on the website.

![CFS Scan Live-feed](../cfsscan-livefeed.png)

Messages are "semi-structured" in terms that they do not have a fixed length or structure. The challenge here is to identify what information is consistently available for analysis.

The screenshot below illustrates the transformation we are aiming to achieve. Semi-structured text messages on the left are programatically parsed to generate the five feature columns on the right.

![Extracted features](../transform-unstructured-text.png)

### Import libraries and sample messages

In [6]:
import re
from collections import Counter
import pandas as pd

In [7]:
# import the first 100 rows
df = pd.read_csv('sample_messages.csv', nrows=100)
df.shape

(100, 4)

In [8]:
# take a look at the data
df.head()

,date time,message id,agency,message
0,02-08-2024 08_41_38,7012498,30,OAK82 PR: 2 - : @ST AGNES SHOPPING CENTRE ST A...
1,02-08-2024 08_41_12,7012496,10,MFS: ADDITIONAL INFO FOR INC024 - FROM STAFF O...
2,02-08-2024 08_41_11,7012495,10,MFS: ADDITIONAL INFO FOR INC024 - FROM STAFF O...
3,02-08-2024 08_41_09,7012494,10,MFS: ADDITIONAL INFO FOR INC024 - FROM STAFF O...
4,02-08-2024 08_41_00,7012493,30,GX77 PR: 7 - : @RAH: FVA ETD 1100 ADELAIDE 3 G...


In [9]:
# check dtypes, column names and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date time   100 non-null    object
 1   message id  100 non-null    int64 
 2   agency      100 non-null    int64 
 3   message     100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


### TODO:
- filter the data to only messages from the "agency" with an id of "30" (Ambulance Service)
- reformat the "date time" column to datetime dtype
- rename the "message id" column to just "id"
- inspect a sample of messages and determine what info can be extracted

In [11]:
# filter results to only agency "30"
df = df[df['agency'] == 30]

# reformat the date time column
df['datetime'] = pd.to_datetime(df['date time'], format="%d-%m-%Y %H_%M_%S")

# rename the message id column
df = df.rename(columns={'message id': 'id'})

# tidy the columns, drop "agency" as it's no longer required.
df = df[['id', 'datetime', 'message']]

In [12]:
# check the results are as expected
df.head()

,id,datetime,message
0,7012498,2024-08-02 08:41:38,OAK82 PR: 2 - : @ST AGNES SHOPPING CENTRE ST A...
4,7012493,2024-08-02 08:41:00,GX77 PR: 7 - : @RAH: FVA ETD 1100 ADELAIDE 3 G...
5,7012492,2024-08-02 08:40:02,WK71 PR: 2 - WAIKERIE D00346 Disp: 08:39 Chest...
6,7012489,2024-08-02 08:40:02,WK71 PR: 2 - WAIKERIE D00346 Disp: 08:39 Chest...
11,7012482,2024-08-02 08:34:02,MV62 Crew Welfare Check URGMSG


### Inspect the messages, identify patterns

In [14]:
# print the first five messages in full and look for patterns
for msg in df['message'][:5]:
    print(msg)

OAK82 PR: 2 - : @ST AGNES SHOPPING CENTRE ST AGNES 85 C 12 D00352 Disp: 08:41 Unconsciou
GX77 PR: 7 - : @RAH: FVA ETD 1100 ADELAIDE 3 G 1 D00348 Disp: 08:40 TRANSFER
WK71 PR: 2 - WAIKERIE D00346 Disp: 08:39 Chest Pain
WK71 PR: 2 - WAIKERIE D00346 Disp: 08:39 Chest Pain
MV62 Crew Welfare Check URGMSG


---

__Upon closer inspection of the above messages, a few patterns begin to emerge...__

1) Every message starts with a 4 or 5 digit alpha-numeric code ("OAK82", "GX77", "WK71"). Seems like this could be the __"unit"__ being dispatched to the job.
2) Most messages include a reference to "PR:" followed by a number. This is likely to be a reference to __"priority"__, which is an important feature for analysis.
3) Most messages seem to reference a __suburb or location__ for the job (eg "ST AGNES", "ADELAIDE", "WAIKERIE"), another important feature for analysis.
4) All messages which include a reference to "priority"/"PR", seem to end with a __description__ of the job (eg "Unconsciou[s]", "TRANSFER", "Chest Pain")
5) Messages without a reference to "priority"/"PR" don't seem to be related to a specific job.
6) Messages with an "@" symbol seem to be public places or buildings
7) Alternatively, messages without the "@" symbol seem likely to be for __private/residential__ addresses.

---

### TODO:
- Check assumption about point 5 above - filter out if it holds true
- Extract the "unit" identifier.
- Extract the "priority" level.
- Extract the job "description"
- Indicate if job "location" is a residential or public place
- Extract the job "location" (suburb)

In [18]:
# filter the dataframe to only those messages which don't include "PR:" - can we drop these records? Yes.
non_pr = df[~df['message'].str.contains('PR:')]
non_pr

,id,datetime,message
11,7012482,2024-08-02 08:34:02,MV62 Crew Welfare Check URGMSG
28,7012461,2024-08-02 08:23:23,Please call CSDNO
29,7012460,2024-08-02 08:22:47,SEA63 Crew Welfare Check URGMSG
31,7012458,2024-08-02 08:20:17,HEAD TO EW TO PICK UP YOUR PARTNER
32,7012456,2024-08-02 08:14:23,CAN YOU PLEASE TRACK TO PARAFIELD FOR STANDBY-...
34,7012454,2024-08-02 08:13:15,CAN YOU PLEASE TRACK TO REDWOOD PARK FOR STAND...
51,7012436,2024-08-02 07:53:45,CREW P2 FROM LOCKLEYS AREA


In [19]:
# assumption from point 5 holds true, these are non-job related messages and can be safely dropped
df = df[df['message'].str.contains('PR:')].reset_index(drop=True)

# extract the "unit" identifier
df['unit'] = df['message'].apply(lambda x: x.split(' ')[0])

# extract the "priority" level
df['priority'] = df['message'].apply(lambda x: x.split('-')[0].split('PR:')[-1].strip())

# residential or public "location" boolean
df['residential'] = ~df['message'].str.contains('@')

# extract the job "description" (find the "dispatch time" and extract the text that follows)
df['description'] = df['message'].str.split(r'\b\d\d:\d\d\b', regex=True).apply(lambda x: x[-1])


### Extract job location from message

The final step is to determine what suburb/town is mentioned in the message. To do this, we will take a list of South Australian suburbs to match with locations mentioned in the messages.

A few things to be careful of here: 
1) Exact matching the suburb - eg, "ADELAIDE" vs "NORTH ADELAIDE".
2) Punctuation - eg, "O'HALLORAN HILL" and "OHALLORAN HILL".
3) Conflicting locations - In the case where multiple suburbs are matched in a message (only non-residential locations) - the suburb/location with the most matches will be chosen. eg - "@GLYNDE Aged Care CAMPBELLTOWN - CAMPBELLTOWN" will be matched with "CAMPBELLTOWN"

In [21]:
# import the list of South Australian suburbs
suburbs = pd.read_csv('SA_suburbs.csv', usecols=['suburb'])['suburb'].tolist()

In [22]:
# check the last few suburbs in the list
suburbs[-5:]

['MULOORINA', 'MUNDOWDNA', 'MUNGERANIE', 'CROWN POINT', 'ADELAIDE AIRPORT']

In [23]:
# function to account for alternative patterns in punctuation
def create_alternative_patterns(suburb):
    suburb_escaped = re.escape(suburb)
    suburb_no_specials = re.escape(re.sub(r'[^A-Za-z0-9\s]', '', suburb))
    return f'{suburb_escaped}|{suburb_no_specials}'

In [24]:
# Compile the pattern with word boundaries to exact match whole words
suburb_pattern = '|'.join(create_alternative_patterns(suburb) for suburb in sorted(suburbs, key=len, reverse=True))

suburb_regex = re.compile(r'\b(' + suburb_pattern + r')\b', re.IGNORECASE)

In [25]:
# function to extract/match the suburb from the message
def find_suburb(message, suburb_regex):
    
    matches = suburb_regex.findall(message)
    if matches:
        # Count occurrences of each matched suburb
        match_counts = Counter(matches)
        # Return the suburb with the highest count
        return match_counts.most_common(1)[0][0]
    return None

# apply the above function to all messages
df['matched_suburb'] = df['message'].apply(find_suburb, suburb_regex=suburb_regex)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              72 non-null     int64         
 1   datetime        72 non-null     datetime64[ns]
 2   message         72 non-null     object        
 3   unit            72 non-null     object        
 4   priority        72 non-null     object        
 5   residential     72 non-null     bool          
 6   description     72 non-null     object        
 7   matched_suburb  72 non-null     object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 4.1+ KB


In [27]:
# take a look at the frame to ensure everything looks good
df.head()

,id,datetime,message,unit,priority,residential,description,matched_suburb
0,7012498,2024-08-02 08:41:38,OAK82 PR: 2 - : @ST AGNES SHOPPING CENTRE ST A...,OAK82,2,False,Unconsciou,ST AGNES
1,7012493,2024-08-02 08:41:00,GX77 PR: 7 - : @RAH: FVA ETD 1100 ADELAIDE 3 G...,GX77,7,False,TRANSFER,ADELAIDE
2,7012492,2024-08-02 08:40:02,WK71 PR: 2 - WAIKERIE D00346 Disp: 08:39 Chest...,WK71,2,True,Chest Pain,WAIKERIE
3,7012489,2024-08-02 08:40:02,WK71 PR: 2 - WAIKERIE D00346 Disp: 08:39 Chest...,WK71,2,True,Chest Pain,WAIKERIE
4,7012480,2024-08-02 08:31:23,OAK820 PR: 3 - SALISBURY EAST 71 P 9 D00339 Di...,OAK820,3,True,Abdominal,SALISBURY EAST


### Wrap it all in a single function and test with all 2000 rows

In [29]:
# helper function to account for alternative patterns in suburb punctuation
def create_alternative_patterns(suburb):
    suburb_escaped = re.escape(suburb)
    suburb_no_specials = re.escape(re.sub(r'[^A-Za-z0-9\s]', '', suburb))
    return f'{suburb_escaped}|{suburb_no_specials}'


# helper function to extract/match the suburb from the message
def find_suburb(message, suburb_regex):
    
    matches = suburb_regex.findall(message)
    if matches:
        # Count occurrences of each matched suburb
        match_counts = Counter(matches)
        # Return the suburb with the highest count
        return match_counts.most_common(1)[0][0]
    return None

# main wrapper function
def import_extract_saas_messages(filepath) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    
    # filter results to only agency "30" ["South Australian Ambulance Service/SAAS"]
    df = df[df['agency'] == 30]

    # reformat the date time column
    df['datetime'] = pd.to_datetime(df['date time'], format="%d-%m-%Y %H_%M_%S")

    # rename the message id column
    df = df.rename(columns={'message id': 'id'})

    # tidy the columns, drop "agency" as it's no longer required.
    df = df[['id', 'datetime', 'message']]
    
    # non-job related messages can be safely dropped
    df = df[df['message'].str.contains('PR:')].reset_index(drop=True)

    # extract the "unit" identifier
    df['unit'] = df['message'].apply(lambda x: x.split(' ')[0])

    # extract the "priority" level
    df['priority'] = df['message'].apply(lambda x: x.split('-')[0].split('PR:')[-1].strip())

    # residential or public "location" boolean
    df['residential'] = ~df['message'].str.contains('@')

    # extract the job "description" (find the "dispatch time" and extract the text that follows)
    df['description'] = df['message'].str.split(r'\b\d\d:\d\d\b', regex=True).apply(lambda x: x[-1])
    
    # import the list of South Australian suburbs
    suburbs = pd.read_csv('SA_suburbs.csv', usecols=['suburb'])['suburb'].tolist()
    
    # Compile the pattern with word boundaries to exact match whole words
    suburb_pattern = '|'.join(create_alternative_patterns(suburb) for suburb in sorted(suburbs, key=len, reverse=True))
    suburb_regex = re.compile(r'\b(' + suburb_pattern + r')\b', re.IGNORECASE)
    
    # apply the matching function to all messages
    df['matched_suburb'] = df['message'].apply(find_suburb, suburb_regex=suburb_regex)
    
    return df
    

In [30]:
# test the function on all 2000 rows, then export as CSV to eyeball the results
df = import_extract_saas_messages(filepath='sample_messages.csv')
df.tail(10)

,id,datetime,message,unit,priority,residential,description,matched_suburb
1313,7009756,2024-07-31 19:36:00,PFD192 PR: 2 - RIVERLEA PARK 38 K 1 D01261 Dis...,PFD192,2,True,Burns (Sca,RIVERLEA PARK
1314,7009749,2024-07-31 19:29:40,N182 PR: 3 - : @SAPOL - STURT POLICE STATION: ...,N182,3,False,OTHER EMER,STURT
1315,7009745,2024-07-31 19:27:38,BE171 PR: 6 - BERRI SBI 1 E 6 D01253 Disp: 19:...,BE171,6,True,Psychiatri,BERRI
1316,7009741,2024-07-31 19:26:49,G171 PR: 2 - GAWLER EAST D01252 Disp: 19:26 Br...,G171,2,True,Breathing,GAWLER EAST
1317,7009740,2024-07-31 19:26:47,P192 PR: 2 - BROMPTON 105 N 15 D01251 Disp: 19...,P192,2,True,Unconsciou,BROMPTON
1318,7009737,2024-07-31 19:25:46,MG171 PR: 2 - : @EUREKA VILLAGE - MOUNT GAMBIE...,MG171,2,False,Chest Pain,MOUNT GAMBIER
1319,7009736,2024-07-31 19:25:40,F184 PR: 2 - : H712 1ST MILLSWOOD 130 C 10 D01...,F184,2,True,Breathing,MILLSWOOD
1320,7009735,2024-07-31 19:25:40,F184 PR: 2 - : H712 1ST MILLSWOOD 130 C 10 D01...,F184,2,True,Breathing,MILLSWOOD
1321,7009731,2024-07-31 19:23:19,GR192 PR: 2 - : @REGIS PLAYFORD - DAVOREN PARK...,GR192,2,False,Unconsciou,DAVOREN PARK
1322,7009730,2024-07-31 19:22:44,MV182 PR: 2 - CHRISTIE DOWNS 186 A 4 D01244 Di...,MV182,2,True,Unconsciou,CHRISTIE DOWNS


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1323 non-null   int64         
 1   datetime        1323 non-null   datetime64[ns]
 2   message         1323 non-null   object        
 3   unit            1323 non-null   object        
 4   priority        1323 non-null   object        
 5   residential     1323 non-null   bool          
 6   description     1323 non-null   object        
 7   matched_suburb  1322 non-null   object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 73.8+ KB


In [32]:
# take a look at the message which didn't match a suburb (one miss from 1,300+ rows is acceptable and can be dealt with at the analysis stage)
df[df['matched_suburb'].isnull()]

,id,datetime,message,unit,priority,residential,description,matched_suburb
1155,7010126,2024-08-01 00:17:53,PG171 PR: 7 - : @RV POINT - ROOPENA: RV WY171 ...,PG171,7,False,TRANSFER,None


In [33]:
#df.to_csv('sample_extraction.csv', index=False)